In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import scipy.sparse as sp
import tensorflow as tf
from datetime import datetime as dt

**Load data**

In [2]:
n2_data = np.load("n2_data.npz", allow_pickle=True)
data = n2_data["arr_0"].tolist()
N_u = 30

**Transform C**

In [3]:
C_all = data["C"]
C_all = np.reshape(C_all, (1200, 2, 5, 10))
C_all = np.transpose(C_all, (0, 3, 1, 2))

**Transform R**

In [4]:
R_all = data["R"]
R_all = np.repeat(R_all, repeats=5, axis=1)
C_mult = np.abs(data["C"])
R_all.shape
C_mult.shape
R_all_mean = np.einsum("nam,nar->nmr", C_mult, R_all) / len(C_all[0])
R_all = data["R"][:, None, :, :] - R_all_mean[:, :, None, :]

**Transform S**

In [5]:
S_all = data["S"]
S_all = S_all.reshape((1200, 2, 5, 2, 5))
filter = np.array([
    [1., 1., 0.],
    [1., 1., 0.],
    [0., 0., 0.]
])
filter2 = np.array([
    [0., 0., 0.],
    [0., 0., 0.],
    [0., 0., 1.]
])
S_s_part = S_all[:, :, :3, :, :3] * filter[None, None, :, None, :]
S_p_part = np.einsum("nbaca->nbc", S_all[:, :, 2:, :, 2:])
S_p_part = S_p_part[:, :, None, :, None] * filter2[None, None, :, None, :]
S_all = S_s_part + S_p_part

**Transform H1**

In [6]:
# TODO: validate
H1_all = data["H1"]
C = data["C"]
H1_all = np.einsum("nba,nbe,nef->naf", C, H1_all, C)
H1_all = np.kron(np.eye(2), H1_all)
a, b, c = np.nonzero(H1_all)
h1_idx = np.stack([b, c], axis=1)
h1_idx = np.reshape(h1_idx, (1200, 200, 2))

**Transform H2**

In [7]:
H2_all = data["H2"]
C = data["C"]
H2_all = np.einsum("nba,nxbey,nef->nxafy", C, H2_all, C)
H2_all = np.einsum("nba,nbxye,nef->naxyf", C, H2_all, C)
H2_all = np.kron(np.eye(2)[None, :, :, None, None] * np.eye(2)[None, None, None, :, :], H2_all)
a, b, c, d, e = np.nonzero(H2_all)
h2_idx = np.stack([b, c, d, e], axis=1)
h2_idx = np.reshape(h2_idx, (1200, 40000, 4))

**MO neighbours**

In [13]:
mo_neighbours = data["H2"]
mo_neighbours = np.einsum("nabcd,ab,cd->nabcd", mo_neighbours, np.eye(10), np.eye(10))
mo_neighbours = np.einsum("nabcd->nac", mo_neighbours)

**Create Z**

In [14]:
Z_n2 = np.array([0, 1, 2, 2, 2])
Z_n2 = np.stack([Z_n2, Z_n2], axis=0)
Z_all = np.stack([Z_n2 for _ in range(1200)], axis=0)

**Create V_n**

In [15]:
V_n = np.arange(10, dtype=np.float32) < 7
V_n = np.stack([V_n, V_n], axis=0)
V_n = np.repeat(V_n[None], repeats=1200, axis=0)

In [16]:
n2_data_preprocessed = {
    "Z": Z_all,
    "R": R_all,
    "S": S_all,
    "C": C_all,
    "h1": H1_all,
    "h1_idx": h1_idx,
    "h2": H2_all,
    "h2_idx": h2_idx,
    "V_n": V_n,
    "mo_neighbours": mo_neighbours
}
np.savez("n2_data_preprocessed.npz", n2_data_preprocessed)